In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from pyproj import Geod
#from arcdist import arcdist

#try:
#    from pyproj import Geod
#except:
#    !pip install pyproj
#    from pyproj import Geod

plt.rcParams['figure.figsize'] = 10, 6

# Adapted from compute_Gik_ray_template.m for Applied Seismology, GEOS 626, Carl Tape
# Coding by Amanda McPherson, March 2021

# Template script for computing an element of the partial derivatives matrix, Gik

In [ ]:
# Axis limits and constants

ax1 = [-120.157113, -114.809623, 32., 36.364429]
lonmin = ax1[0]
lonmax = ax1[1]
latmin = ax1[2]
latmax = ax1[3]
earthr = 6371*1e3      # earth radius, in meters
deg = 180/np.pi

# REFERENCE HOMOGENEOUS PHASE VELOCITY
c0 = 3500      # m/s

# geoid for calculating great circle path
#geoid = Geod(ellps="WGS84")
geoid = Geod(ellps="sphere")

In [ ]:
# Load data

# sources
slon, slat, sind = np.loadtxt('./data/events_lonlat.dat',unpack=True,skiprows=1)
nsrc = len(slat)

# receivers
rlon, rlat, rind = np.loadtxt('./data/recs_lonlat.dat',unpack=True,skiprows=1)
nrec = len(rlat)

# spline centers
clon, clat = np.loadtxt('./data/con_lonlat_q08.dat',unpack=True)
nspline = len(clat)

In [ ]:
# Plot the data

plt.scatter(clon,clat,s=3)
for num in range(1,nspline+1):
    plt.text(clon[num-1],clat[num-1],num,size=8)

plt.axis(ax1)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Center-points of spherical spline basis functions')
plt.show()

In [ ]:
# helper function that acts like linspace for a great-circle ray path
# numpts is the total number of points on the ray path, including the start and end points
# npts is the method of the geoid object
# type geoid.npts? for details
# example for npts(): lonlats = geoid.npts(slon, slat, rlon, rlat, 12)
def geoid_linspace(lon_start, lat_start, lon_end, lat_end, numpts):
    if numpts < 2:
        raise ValueError('Not allowed!')
    dist_m = geoid.inv(lon_start, lat_start, lon_end, lat_end)[2]
    lons, lats = np.array([(lon_start, lat_start)] + geoid.npts(lon_start, lat_start, lon_end, lat_end, numpts - 2) + [(lon_end, lat_end)]).T
    return lons, lats, dist_m

In [ ]:
# examples
lon_start = -118
lat_start = 33
lon_end = -116
lat_end = 36
numpts = 5
lons, lats, _dist_m = geoid_linspace(lon_start, lat_start, lon_end, lat_end, numpts)
print(lons.shape)
print(lats.shape)

# necessary commands in order to use lons and lats in Bkspline.py
lons = np.atleast_2d(lons).T
lats = np.atleast_2d(lats).T
print(lons.shape)
print(lats.shape)
print('longitude         latitude')
print(np.concatenate((lons,lats),axis=1))

In [ ]:
# comparison for different geoids (using the sphere is fine for our purposes)

geoid = Geod(ellps="WGS84")
lons, lats, dist_m = geoid_linspace(lon_start, lat_start, lon_end, lat_end, numpts)
print('distance in km:',dist_m/1000)
print('angular distance (assuming a sphere):',dist_m/earthr*deg)

geoid = Geod(ellps="sphere")
lons, lats, dist_m = geoid_linspace(lon_start, lat_start, lon_end, lat_end, numpts)
print('distance in km:',dist_m/1000)
print('angular distance (assuming a sphere):',dist_m/earthr*deg)

In [ ]:
# THIS SHOWS HOW THE MEASUREMENTS (AND ROWS OF MATRIX G) ARE INDEXED
if False:
    print('nsrc = ',nsrc)
    print('nrec = ',nrec)
    print('nmeas = ',nsrc*nrec)
    print('     i  isrc irec')
    for isrc in range(1,nsrc+1):         # ranges 1 to nsrc
        for irec in range(1,nrec+1):     # ranges 1 to nrec
            # measurement index (ith row of matrix G); ranges 1 to nsrc*nrec
            i = (isrc-1)*nrec + irec
            print('%6i%6i%6i'% (i,isrc,irec))  

In [ ]:
# compute the design matrix

# scale of spherical spline basis function
q = 8

# number of points along each ray path, including the endpoints
numpts = 1000

# number of measurements (one per station or ray)
ndata = nrec*nsrc

# initialize the partial derivatives matrix
Gik = np.zeros((ndata,nspline))

# FILL THE ENTRIES OF THE PARTIAL DERIVATIVES MATRIX
# USE THE MEASUREMENT INDEX FOR LOOPS AS A GUIDE
